### The notebook is used to download planetscope images with custom requiremnets (Kehan Yang, kyang33@uw.edu)

#### Load packages and set up directories

In [1]:
import glob
from get_planet import *
from os.path import exists
import os

# directory for the geomtry, the format has to be geojson
dir_geom = "/Users/kehanyang/Documents/resarch/pc2_meadows/data/snotel/meadow_selected_extent_geojson_download2/"
dir_geom = '../../Alaska_project/data/GIS/'
# directory where the images will be stored. 
dir_download = "/Users/kehanyang/Documents/resarch/pc2_meadows/data/planet/orders/Meadows/"
# dir_meadow_images = "/Users/kehanyang/Documents/resarch/pc2_meadows/data/planet/orders/Meadows/"
#
dir_order_url =  '../../Alaska_project/data/planet/'

# change the flag if search and/or download data are required.
flag_search = True
flag_download = True


# 
ID_period = '2017'
#file to store url -- planet data download links
file_orders = dir_order_url+ID_period+'.txt'

/home/kyang33/.virtualenvs/planet_project/lib/python3.8/site-packages/planet/api/__init__.py:38: ClientV1DeprecationWarning: The planet.api module is deprecated and will be removed in version 2.0.0. For more details please see the discussion at https://github.com/planetlabs/planet-client-python/discussions.
  warnings.warn(


Setup OK: API key valid


In [ ]:
# Import the os module in order to access environment variables
import os

# If you're following along with this notebook, you can enter your API Key on the following line, and uncomment it:
#os.environ['PL_API_KEY']='YOUR API KEY HERE'

# Setup the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

In [2]:
if flag_search:
    start_time = ID_period + '-01-01T00:00:00'
    # end_time = '2021-12-31T12:00:00'
    end_time = ID_period + '-12-31T12:00:00'
    overlap = 99 # at least with 99% overlap 
    cloud_pct = 0.05 # no more than 5% cloud cover
     
    # no light haze/heavy haze
    light_haze = 0
    heavy_haze = 0
    
    #search for geojson file
    fn = glob.glob(dir_geom + "*geojson")
    ID_shp = [id.split("/")[-1] for id in fn]
    df = pd.DataFrame(data = {
        "file": fn, 
        "index":  [i.split("/")[-1] for i in fn],
        "ID": [id.split("/")[-1].split('.')[0] for id in fn]
        })
    df = df.sort_values("index", ascending = True)

    print(df.head())

# check whether the order url txt file is exist. if exist, read data; otherwise, creat file.
    idx = 0 
    if exists(file_orders):
        order_urls = pd.read_csv(file_orders)
    else:
        order_urls = pd.DataFrame(columns = {"index","ID_geom", "order_url"})


    for irow in df.itertuples():
    
    # Search id 
        
        print(irow)
        ID_geom = irow.ID.split(".")[0]+ '_' + ID_period
        print(ID_geom)

        if ID_geom not in order_urls.ID_geom.to_list():

            print('Searching available images ------- ')
            idlist = ft_iterate(item_type='PSScene', # planet has changed the product item type from 'PSScene4Band' with PSScene
                    asset_type= 'analytic_sr',
                    geom = read_geom(irow.file),#".json"),
                    start = start_time,
                    end = end_time,
                    cloud_cover = cloud_pct, #cloud cover range 0-1 represting 0-100% so 0.5 means max allowed 50% cloud cover
                    ovp = overlap) #% minimum % overlap 0-100
            
                    # light_haze_percent = light_haze,
                    # heavy_haze_percent = heavy_haze

            print(idlist.shape)
            idlist.sort_values("date")
            idlist.to_csv(dir_meadow_images+ID_geom+'.csv')
            
            # print(irow.file)
            payload_info = order_payload(Name_download = ID_geom, ID_imgs = idlist.id.values.tolist(), File_geom = irow.file)
            # print(payload_info)
            print("Pay order:".format(),ID_geom)


            order_url = order_now(payload_info) # error response 400  

            order_urls.loc[idx, "index"] = idx        
            order_urls.loc[idx, "ID_geom"] = ID_geom
            order_urls.loc[idx, "order_url"] = order_url


            # order_urls.append(order_url)  # save all URLs
            order_urls.to_csv(file_orders, index = None)# save all URLs

            
        idx = idx + 1



                                                file  \
0  ../../Alaska_project/data/GIS/Utqiagvik_site_b...   

                             index                       ID  
0  Utqiagvik_site_boundary.geojson  Utqiagvik_site_boundary  
Pandas(Index=0, file='../../Alaska_project/data/GIS/Utqiagvik_site_boundary.geojson', index='Utqiagvik_site_boundary.geojson', ID='Utqiagvik_site_boundary')
Utqiagvik_site_boundary_2017
Searching available images ------- 


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

In [4]:
from planet import api
client = api.ClientV1()


In [34]:
 idlist = api.filters.range_filter(item_type='PSScene', # planet has changed the product item type from 'PSScene4Band' with PSScene
                    asset_type= 'analytic_sr',
                    geom = read_geom(irow.file),#".json"),
                    start = start_time,
                    end = end_time,
                    cloud_cover = cloud_pct, #cloud cover range 0-1 represting 0-100% so 0.5 means max allowed 50% cloud cover
                    ovp = overlap,
                    light_haze_percent = light_haze,
                    heavy_haze_percent = heavy_haze) #% minimum % overlap 0-100

TypeError: range_filter() missing 1 required positional argument: 'field_name'

In [5]:
# build a filter for the AOI
filter = api.filters.range_filter("clear_percent", gte=90)

# show the structure of the filter
print(filter)

{'field_name': 'clear_percent', 'type': 'RangeFilter', 'config': {'gte': 90}}


In [13]:
api.filters.build_search_request

<function planet.api.filters.build_search_request(filter_like, item_types, name=None, interval=None)>

In [6]:
# we are requesting PlanetScope 4 Band imagery
item_types = ['PSScene']
request = api.filters.build_search_request(filter, item_types)

# this will cause an exception if there are any API related errors
results = client.quick_search(request)

In [7]:
results

In [8]:
# print out the ID of the most recent 10 images that matched
for item in results.items_iter(10):
    print('%s' % item['id'])

20230505_054622_62_2402
20230505_054618_56_2402
20200423_073226_09_2275
20230505_080533_83_2439
20230505_080506_45_2439
20230505_071439_35_248e
20230505_103112_35_247a
20230505_102555_40_247a
20230505_080538_04_2439
20230505_080508_56_2439
